In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import keras.backend as K

IMAGE_SHAPE = [94,24]
CHARS = "ABCDEFGHIJKLMNPQRSTUVWXYZ0123456789" # exclude I, O
CHARS_DICT = {char:i for i, char in enumerate(CHARS)}
DECODE_DICT = {i:char for i, char in enumerate(CHARS)}
NUM_CLASS = len(CHARS)+1



In [2]:
def CTCLoss(y_true, y_pred):
    # Compute the training-time loss value
    batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
    input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
    label_length = tf.cast(tf.shape(y_true)[1], dtype="int64")

    input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int64")
    label_length = label_length * tf.ones(shape=(batch_len, 1), dtype="int64")

    loss = keras.backend.ctc_batch_cost(y_true, y_pred, input_length, label_length)
    
    return loss

In [3]:
class small_basic_block(keras.layers.Layer):

    def __init__(self,out_channels,name=None,**kwargs):
        super().__init__(**kwargs)
        out_div4=int(out_channels/4)
        self.main_layers = [
            keras.layers.SeparableConv2D(out_div4,(1,1),padding='same',kernel_initializer=keras.initializers.glorot_uniform(),bias_initializer=keras.initializers.constant()),
            keras.layers.BatchNormalization(),
            keras.layers.ReLU(),
            keras.layers.SeparableConv2D(out_div4,(3,1),padding='same',kernel_initializer=keras.initializers.glorot_uniform(),bias_initializer=keras.initializers.constant()),
            keras.layers.BatchNormalization(),
            keras.layers.ReLU(),
            keras.layers.SeparableConv2D(out_div4,(1,3),padding='same',kernel_initializer=keras.initializers.glorot_uniform(),bias_initializer=keras.initializers.constant()),
            keras.layers.BatchNormalization(),
            keras.layers.ReLU(),
            keras.layers.SeparableConv2D(out_channels,(1,1),padding='same',kernel_initializer=keras.initializers.glorot_uniform(),bias_initializer=keras.initializers.constant()),
            keras.layers.BatchNormalization(),
            keras.layers.ReLU(),
        ]  
    
    def call(self,input):
        x = input
        for layer in self.main_layers:
            x = layer(x)
        return x

In [4]:
#test this later

class global_context(keras.layers.Layer):
    def __init__(self,kernel_size,stride,**kwargs):
        super().__init__(**kwargs)
        self.ksize = kernel_size
        self.stride = stride


    def call(self, input):
        x = input 
        avg_pool = keras.layers.AveragePooling2D(pool_size=self.ksize,strides=self.stride,padding='same')(x)
        sq = keras.layers.Lambda(lambda x: tf.math.square(x))(avg_pool)
        sqm = keras.layers.Lambda(lambda x: tf.math.reduce_mean(x))(sq)
        out = keras.layers.Lambda(lambda x: tf.math.divide(x[0], x[1]))([avg_pool , sqm])
        #out = keras.layers.Lambda(lambda x: K.l2_normalize(x,axis=1))(avg_pool)
        return out

In [5]:
class LPRnet(keras.Model):
    def __init__(self, input_shape=(24,94,3), **kwargs):
        super(LPRnet, self).__init__(**kwargs)
        self.input_layer = keras.layers.Input(input_shape)
        self.cnn_layers= [
            keras.layers.SeparableConv2D(64,kernel_size = (3,3),strides=1,padding='same',name='main_conv1',kernel_initializer=keras.initializers.glorot_uniform(),bias_initializer=keras.initializers.constant()),
            keras.layers.BatchNormalization(name='BN1'),
            keras.layers.ReLU(name='RELU1'),
            keras.layers.MaxPool2D(pool_size=(3,3),strides=(1,1),name='maxpool2d_1',padding='same'),
            small_basic_block(128),
            keras.layers.MaxPool2D(pool_size=(3,3),strides=(1,2),name='maxpool2d_2',padding='same'),
            small_basic_block(256),
            small_basic_block(256),
            keras.layers.MaxPool2D(pool_size=(3,3),strides=(1,2),name='maxpool2d_3',padding='same'),
            keras.layers.Dropout(0.5),
            keras.layers.SeparableConv2D(256,(4,1),strides=1,padding='same',name='main_conv2',kernel_initializer=keras.initializers.glorot_uniform(),bias_initializer=keras.initializers.constant()),
            keras.layers.BatchNormalization(),
            keras.layers.ReLU(),
            keras.layers.Dropout(0.5),
            keras.layers.SeparableConv2D(NUM_CLASS,(1,13),padding='same',name='main_conv3',kernel_initializer=keras.initializers.glorot_uniform(),bias_initializer=keras.initializers.constant()),  
            keras.layers.BatchNormalization(),
            keras.layers.ReLU(),
        ]
        self.out_layers = [
            keras.layers.SeparableConv2D(NUM_CLASS,kernel_size=(1,1),strides=(1,1),padding='same',name='conv_out',kernel_initializer=keras.initializers.glorot_uniform(),bias_initializer=keras.initializers.constant()),
            #keras.layers.BatchNormalization(),
            #keras.layers.ReLU(),
        ]
        #self.conv_out = keras.layers.Conv2D(NUM_CLASS,kernel_size=(1,1),strides=(1,1),padding='same',name='conv_out')
        self.out = self.call(self.input_layer)
        super(LPRnet, self).__init__(
            inputs=self.input_layer,
            outputs=self.out,
            **kwargs)

    def call(self,inputs,training=False):
        x = inputs
        layer_outputs = []
        for layer in self.cnn_layers:
            x = layer(x)
            layer_outputs.append(x)
        scale1 = global_context((1,4),(1,4))(layer_outputs[0])
        scale2 = global_context((1,4),(1,4))(layer_outputs[4])
        scale3 = global_context((1,2),(1,2))(layer_outputs[6])
        scale5 = global_context((1,2),(1,2))(layer_outputs[7])
        #scale4 = keras.layers.Lambda(lambda x: K.l2_normalize(x,axis=1))(x)
        sq = keras.layers.Lambda(lambda x: tf.math.square(x))(x)
        sqm = keras.layers.Lambda(lambda x: tf.math.reduce_mean(x))(sq)
        scale4 = keras.layers.Lambda(lambda x: tf.math.divide(x[0], x[1]))([x , sqm])
        gc_concat = keras.layers.Lambda(lambda x: tf.concat([x[0], x[1], x[2], x[3], x[4]],3))([scale1, scale2, scale3, scale5,scale4])
        for layer in self.out_layers:
            gc_concat = layer(gc_concat)
        logits = keras.layers.Lambda(lambda x: tf.math.reduce_mean(x[0],axis=1))([gc_concat])
        #transposed_logs = keras.layers.Lambda(lambda x: tf.transpose(x,(1,0,2)))(logits)
        logits = keras.layers.Softmax()(logits)
        return logits

In [6]:
model = LPRnet()
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=2000,
    decay_rate=0.9
)

In [12]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),loss =CTCLoss)
model.summary()

Model: "lp_rnet_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 24, 94, 3)]  0                                            
__________________________________________________________________________________________________
main_conv1 (SeparableConv2D)    (None, 24, 94, 64)   283         input_1[0][0]                    
__________________________________________________________________________________________________
BN1 (BatchNormalization)        (None, 24, 94, 64)   256         main_conv1[0][0]                 
__________________________________________________________________________________________________
RELU1 (ReLU)                    (None, 24, 94, 64)   0           BN1[0][0]                        
__________________________________________________________________________________________

In [13]:
from gen_plates_keras import *
gen = ImageGenerator()
def generate_dataset(num = 100):
    data, labels = gen.generate_images(num)
    gen_labels = []
    for label in labels:
        gen_labels.append([CHARS_DICT[i] for i in label.split('_')[0]])
    pics =np.array(data)
    labels = np.array(labels)
    training_set = np.array(pics,dtype=np.float32)
    training_labels = np.array(gen_labels)
    ragged = tf.ragged.constant(training_labels).to_tensor()
    dataset = tf.data.Dataset.from_tensor_slices((training_set,ragged)).shuffle(640).batch(64)
    return dataset

standalone_dataset = generate_dataset(num=100)

C:\Users\carlos\AppData\Local\Temp/ipykernel_2264/520954477.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training_labels = np.array(gen_labels)


In [14]:
import wandb
from wandb.keras import WandbCallback
wandb.init(project="LPRnet-separableconv2d", entity="clsandoval")
wandb.config = {
  "learning_rate": 0.001,
  "epochs": 400,
  "batch_size": 64
}

def generate_val_set(num=400):
        val_data_gen, val_labels_gen = gen.generate_images(num)
        val_labels_encoded = []
        val_data_encoded = []
        for image,label in zip(val_data_gen, val_labels_gen):
                val_data_encoded.append(np.expand_dims(image,axis=0))
                val_labels_encoded.append([CHARS_DICT[i] for i in label.split('_')[0]])
        return val_data_encoded,val_labels_encoded
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss',patience=10)

In [15]:
val_dataset = generate_dataset(1000)
for j in range(80):
    dataset = generate_dataset(num=32000)
    model.fit(dataset,validation_data=val_dataset,validation_steps=5,epochs=10,steps_per_epoch=50,callbacks=[WandbCallback()])
    print(j,flush=True)

C:\Users\carlos\AppData\Local\Temp/ipykernel_2264/520954477.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training_labels = np.array(gen_labels)


Epoch 1/10
50/50 [==============================] - 20s 210ms/step - loss: 30.1091 - val_loss: 100.3475


wandb: ERROR Can't save model, h5py returned error: Layer small_basic_block has arguments in `__init__` and therefore must override `get_config`.


Epoch 2/10
50/50 [==============================] - 10s 197ms/step - loss: 22.7080 - val_loss: 100.3633
Epoch 3/10
50/50 [==============================] - 10s 196ms/step - loss: 21.4360 - val_loss: 100.3553
Epoch 4/10
50/50 [==============================] - 10s 196ms/step - loss: 20.4567 - val_loss: 100.3319
Epoch 5/10
50/50 [==============================] - 10s 197ms/step - loss: 20.2516 - val_loss: 99.8105
Epoch 6/10
50/50 [==============================] - 10s 197ms/step - loss: 20.1200 - val_loss: 94.6991
Epoch 7/10
50/50 [==============================] - 10s 198ms/step - loss: 19.7387 - val_loss: 84.7058
Epoch 8/10
50/50 [==============================] - 10s 196ms/step - loss: 19.7991 - val_loss: 68.6289
Epoch 9/10
50/50 [==============================] - 10s 197ms/step - loss: 19.6083 - val_loss: 61.7032
Epoch 10/10
50/50 [==============================] - 10s 198ms/step - loss: 19.4590 - val_loss: 44.4123
0


C:\Users\carlos\AppData\Local\Temp/ipykernel_2264/520954477.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training_labels = np.array(gen_labels)


...

Epoch 9/10
50/50 [==============================] - 12s 235ms/step - loss: 0.5782 - val_loss: 0.5453
Epoch 10/10
50/50 [==============================] - 12s 234ms/step - loss: 0.5518 - val_loss: 0.5507
56
Epoch 1/10
50/50 [==============================] - 12s 230ms/step - loss: 0.5792 - val_loss: 0.5762
Epoch 2/10
50/50 [==============================] - 12s 231ms/step - loss: 0.5420 - val_loss: 0.5961
Epoch 3/10
50/50 [==============================] - 12s 232ms/step - loss: 0.5555 - val_loss: 0.6049
Epoch 4/10
50/50 [==============================] - 12s 232ms/step - loss: 0.5605 - val_loss: 0.6216
Epoch 5/10
50/50 [==============================] - 12s 233ms/step - loss: 0.5604 - val_loss: 0.5095
Epoch 6/10
50/50 [==============================] - 12s 236ms/step - loss: 0.5419 - val_loss: 0.5962
Epoch 7/10
50/50 [==============================] - 12s 234ms/step - loss: 0.5848 - val_loss: 0.5314
Epoch 8/10
50/50 [==============================] - 12s 236ms/step - loss: 0.5249 - val

wandb: ERROR Can't save model, h5py returned error: Layer small_basic_block has arguments in `__init__` and therefore must override `get_config`.


59
Epoch 1/10
50/50 [==============================] - 12s 230ms/step - loss: 0.5697 - val_loss: 0.5794
Epoch 2/10
50/50 [==============================] - 12s 231ms/step - loss: 0.5718 - val_loss: 0.5727
Epoch 3/10
50/50 [==============================] - 12s 233ms/step - loss: 0.5706 - val_loss: 0.6082
Epoch 4/10
50/50 [==============================] - 12s 234ms/step - loss: 0.5275 - val_loss: 0.5510
Epoch 5/10
50/50 [==============================] - 12s 235ms/step - loss: 0.5791 - val_loss: 0.6126
Epoch 6/10
50/50 [==============================] - 12s 231ms/step - loss: 0.5648 - val_loss: 0.5587
Epoch 7/10
50/50 [==============================] - 12s 234ms/step - loss: 0.5308 - val_loss: 0.5296
Epoch 8/10
50/50 [==============================] - 12s 235ms/step - loss: 0.5375 - val_loss: 0.5678
Epoch 9/10
50/50 [==============================] - 12s 236ms/step - loss: 0.5800 - val_loss: 0.5934
Epoch 10/10
50/50 [==============================] - 12s 233ms/step - loss: 0.5206 - val

wandb: ERROR Can't save model, h5py returned error: Layer small_basic_block has arguments in `__init__` and therefore must override `get_config`.


Epoch 10/10
50/50 [==============================] - 12s 236ms/step - loss: 0.5785 - val_loss: 0.5539
69
Epoch 1/10
50/50 [==============================] - 12s 229ms/step - loss: 0.5242 - val_loss: 0.5430
Epoch 2/10
50/50 [==============================] - 12s 231ms/step - loss: 0.5243 - val_loss: 0.4942
Epoch 3/10
50/50 [==============================] - 12s 232ms/step - loss: 0.4865 - val_loss: 0.5062
Epoch 4/10
50/50 [==============================] - 12s 235ms/step - loss: 0.5617 - val_loss: 0.5875
Epoch 5/10
50/50 [==============================] - 12s 234ms/step - loss: 0.5122 - val_loss: 0.4874
Epoch 6/10
50/50 [==============================] - 12s 234ms/step - loss: 0.5097 - val_loss: 0.5855
Epoch 7/10
50/50 [==============================] - 12s 234ms/step - loss: 0.4955 - val_loss: 0.6308
Epoch 8/10
50/50 [==============================] - 12s 234ms/step - loss: 0.5557 - val_loss: 0.6151
Epoch 9/10
50/50 [==============================] - 12s 234ms/step - loss: 0.5407 - val

In [16]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('keras_lprnet_separable2.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\carlos\AppData\Local\Temp\tmpmk9azia4\assets


INFO:tensorflow:Assets written to: C:\Users\carlos\AppData\Local\Temp\tmpmk9azia4\assets


In [3]:
interpreter = tf.lite.Interpreter('keras_lprnet_separable2.tflite')
import numpy as np
import time 
import glob
import cv2

interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
real_images = glob.glob('C:\\Users\\carlos\\Desktop\\cs\\ml-sandbox\\ANPR\\TFODCourse\\test frames\\autolabelled\\*.png')
start = time.time()
for file in real_images:
    image = cv2.imread(file)
    test_image = cv2.resize(image,(94,24))/256
    test_image = np.expand_dims(test_image,axis=0)
    test_image = test_image.astype(np.float32)
    interpreter.set_tensor(input_details[0]['index'], test_image)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    decoded = keras.backend.ctc_decode(output_data,(24,))
    for i in np.array(decoded[0]).reshape(24):
        if i >-1:
            print(DECODE_DICT[i],end='')
    print(" "+ file.split('\\')[-1].split('_')[0])
print(time.time()-start)

AB0S7TQTA AB09710
UAPA2G70 APA2G70
CCAVB710 CAV8710
NAA49B0U NAA4980
NAQB162 NAQ8162
NAW9426 NAW9426
N8J93U0 NBJ9340
NBR99B3 NBR9983
NBX70B3A NBX7083
NC1J1B92 NCA1892
NNCM7694 NCM7694
NC09005 NCU9005
NDC7444A NDC7444
MDL63GAA NDL6364
NDP3545 NDP3545
MNDX975B NDX9758
NDYS5745 NDY5745
NFX837D NFXB370
NGF9B59 NGF9859
ZZRET7LA ZRET74
2.628660202026367
